# Reinforcement Q-learning simple NN as function approximation

- portfolio grid of size 10 (0 to 1)
- 1 hidden neural layers 
- Improved by Dropout 

In [1]:
# Initialization
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

import winsound

In [20]:
data = pd.read_csv('data.csv') #Three stocks (R,X_s,X_b,s_nom,s_pe,s_spr) 

In [17]:
data = pd.read_excel('sim_data_CER.xlsx') #Three stocks (R,X_s,X_b) Without predictors

# Initialization of the Tensorflow placeholders and the network

In [18]:
num_lags = 10
num_stocks=3
num_inputvar = (num_lags+1)*num_stocks
num_actions=10
hidden1 = 30
hidden2 = 20
hidden3 = 15

#Initialize Neural Network and set-up the placeholders
tf.reset_default_graph()
NN_input = tf.placeholder(shape=[1,num_inputvar],dtype=tf.float32)
w1 = tf.Variable(tf.random_uniform([num_inputvar,hidden1],0,1))
w2 = tf.Variable(tf.random_uniform([hidden1,hidden2],0,1))
w3 = tf.Variable(tf.random_uniform([hidden2,hidden3],0,1))
w4 = tf.Variable(tf.random_uniform([hidden3,num_actions],0,1))
b1 = tf.Variable(np.zeros((1,hidden1)), dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,hidden2)), dtype=tf.float32)
b3 = tf.Variable(np.zeros((1,hidden3)), dtype=tf.float32)
b4 = tf.Variable(np.zeros((1,num_actions)), dtype=tf.float32)

h1 = tf.add(tf.matmul(NN_input,w1),b1)
# h_drop1 = tf.nn.dropout(h1,0.5)
a1 = tf.nn.softmax(h1)

h2 = tf.add(tf.matmul(a1,w2),b2)
# h_drop2 = tf.nn.dropout(h2,0.3)
a2 = tf.nn.softmax(h2)

h3 = tf.add(tf.matmul(a2,w3),b3)
# h_drop3 = tf.nn.dropout(h3,0.1)
a3 = tf.nn.softmax(h3)

ol = tf.add(tf.matmul(a3,w4),b4)
output = tf.nn.softmax(ol)

A_Max = tf.argmax(output,1)
# Calculate loss for the NN from the Q values
Q_Next = tf.placeholder(shape=[1,num_actions],dtype=tf.float32)
diff = tf.subtract(Q_Next,output)
loss = tf.multiply(diff,diff)
learning_rate = tf.placeholder(tf.float32, shape=[])
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(a4 * tf.log(Q_Next), reduction_indices=[1]))
optmzr = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(loss)
A = np.linspace(0,1,num_actions) # portfolio weights of stocks (1-weight) is the weight in the bonds

# Training of the NN function approximation

In [ ]:
# initialize
init = tf.global_variables_initializer()
startepsilon = 0.8
endepsilon = 0.01
startlr = 0.8
endlr = 0.01
startgamma = 0.01
endgamma = 0.8
TWlist = []
Index = []
MWeights = []
Turnover = []
chooserlist = []
RU = []
learningrateQ = 0.5
n = 707
periods = 60 
epochs = 50     # preferred to have a low amount of epochs because otherwise the the same data is used multiple times (usually not the case in stock returns)

# data parsing
dates = data['Date']
mdata = data[['r','xs','xb']]
mdata.index = pd.DatetimeIndex(dates)

# include the lags
for i in range(1,num_lags+1):  # 1 till 11
    mdata['r_lag' + str(i)] = mdata['r'].shift(i)
    mdata['xs_lag' + str(i)] = mdata['xs'].shift(i)
    mdata['xb_lag' + str(i)] = mdata['xb'].shift(i)

mdata.drop(mdata.head(num_lags).index, inplace=True)
# have a look at the data (if not include NANs)
# print(mdata)

# Train the the Q-function DNN
with tf.Session() as sess:
    for i in range(339,int(n-periods-1)):
        lostlist = []
        OptimalWeights = np.zeros(periods-1)
        currentK = 0
        print(mdata.index[i])
        rmin = -1
        rmax = 1
        epsilon = 0.15
        lr = 0.3
        gamma = 0.98
        sess.run(init) # initialize the Neural Network again
        currentEpoch = 0
        rlist = []  
        runitlist = []
         
            
        while currentEpoch < epochs:
            indexes = np.asarray(range(i+currentK-1))
            random.shuffle(indexes)
            #Training of the Q-Network for the data available (with Neural Nets) 
            for j in indexes:
                s = mdata.iloc[j,:].values.reshape(1,num_inputvar)
                #Choose an action by greedily (with e chance of random action) from the Q-network
                a_int,allQ = sess.run([A_Max,output],feed_dict={NN_input:s})
                a = A[a_int]  # -1 because index
                if np.random.rand(1) < epsilon:
                    a = random.choice(A)

                #Get new state and reward from environment
                s1 = mdata.iloc[j+1,:].values.reshape(1,num_inputvar)
                r = (a*s1[0][1] + (1-a)*s1[0][2]) #reward: this is now the wealth gained from this step, but could be other rewards like utility
                rlist.append(r)
                Q = sess.run(output,feed_dict={NN_input:s1})
                    
                #Obtain maxQ' and set our target value for chosen action.
                Q1 = np.max(Q)
                targetQ = allQ
                if(len(rlist)>1):
                    rmin = np.min(rlist)
                    rmax = np.max(rlist)
                runit = -1 + 2*((r -rmin)/(rmax-rmin))
#                 targetQ[0,int(a*(num_actions-1))] = targetQ[0,int(a*(num_actions-1))] + learningrateQ*(runit + gamma*Q1 - targetQ[0,int(a*(num_actions-1))]) 
                targetQ[0,int(a*(num_actions-1))] = runit + gamma*Q1
                runitlist.append(runit)
                #Train the neural network using target and predicted Q values
                opt,W1,loss_ = sess.run([optmzr,w1,loss],feed_dict={NN_input:s,Q_Next:targetQ,learning_rate:lr})
                lostlist.append(sum(sum(loss_)))
                print(targetQ)
            currentEpoch += 1
        
        currentK = 0
        while currentK < periods - 1: 
            # After training now calculate the optimal weights for the K=60 periods to come
            s = mdata.iloc[i+currentK,:].values.reshape(1,num_inputvar)
            a_int,allQ = sess.run([A_Max,output],feed_dict={NN_input:s})
            aOpt = A[a_int]
            OptimalWeights[currentK] = aOpt
            currentK += 1
            
        # For insight purposes (Write away)
        plt.ion()
        plt.plot(lostlist)
        plt.show()
        plt.plot(rlist)
        plt.show()
        plt.plot(runitlist)
        plt.show()
        
        firstdiff = OptimalWeights[1:] - OptimalWeights[:-1]
        MWeights.append(np.mean(OptimalWeights))
        TerminalWealth = np.exp(sum(OptimalWeights*mdata[i+1:i+currentK+1]['xs'] + (1-OptimalWeights)*mdata[i+1:i+currentK+1]['xb']))
        TWlist.append(TerminalWealth)
        Index.append(mdata.index[i])
        Turnover.append(sum(abs(firstdiff*np.exp(mdata[i+1:i+currentK]['xs'])) + abs((1-firstdiff)*np.exp(mdata[i+1:i+currentK]['xb']))))
        RU.append((1/(1-5))*pow(TerminalWealth,(1-5)))
        print(TerminalWealth)
        print(OptimalWeights)
        
        print('Writing away results')
        df = pd.DataFrame({'index date':Index,'TW':TWlist, 'Mean Weights Xs':MWeights,'Turnover':Turnover, 'Realized Utility':RU})
        df.to_excel('DNN_4.xlsx', sheet_name='sheet1', index=False)
print('Done!')
        

1983-03-01 00:00:00
[[ 0.0944614   0.09077625  0.10600159  0.10390683  0.15789162  0.08725446
   0.11140069  0.10019571  0.08993888  0.09539902]]
[[ 0.09443223  0.09074838  0.10596296  0.10387006 -0.88145596  0.08722728
   0.11135687  0.10015927  0.08991041  0.09536703]]
[[ 0.09524782  0.09150396  0.10696822  0.10484891  0.11310289  0.08793236
   0.11246128  0.10108118  0.09065615 -0.88915634]]
[[ 0.09576613  0.0919779   0.10763341 -0.88844192  0.11383541  0.08836517
   0.11317655  0.10165651  0.0911219   0.09099576]]
[[ 0.09639741 -0.18541571  0.10842648  0.09930914  0.11472897  0.0889108
   0.1140619   0.10235541  0.09168423  0.09155921]]
[[ 0.09653079  0.09117278  0.10860346  0.09945909  0.51352543  0.08902871
   0.11426317  0.10251637  0.09181086  0.09168748]]
[[ 0.09621251  0.090888    0.10820796  0.09912241 -0.88445443  0.088754
   0.11383317  0.10215843  0.09152064  0.09139901]]
[[ 0.09701556  0.09160569  0.10920497  0.09997545  0.11038261  0.08944315
  -0.88737363  0.10306563  

[[ 0.07056296  0.06762223  0.07747868  0.07338954  0.40171406  0.06834193
   0.0800369   0.07301353  0.06878854  0.06916457]]
[[ 0.07036123  0.06743204  0.07725265  0.07317714  0.85675603  0.06814934
   0.07980051  0.07280188  0.06859455  0.06896782]]
[[ 0.06830885  0.06548913  0.07494531  0.07101631  0.27120122  0.06617935
   0.07739501  0.07065327  0.06660642  0.06696774]]
[[ 0.06873441  0.06589302  0.07542426  0.0714644  -0.30752268  0.06658958
   0.07789395  0.07109962  0.06701938  0.06738331]]
[[ 0.0715156   0.06852569  0.07855171  0.07439393  0.39013231  0.06925789
   0.08115842  0.07401326  0.06971249  0.0700949 ]]
[[ 0.07133159  0.06834822  0.07834364  0.07419825  0.38769329  0.06907934
   0.0809378   0.07381814  0.06953128  0.0699124 ]]
[[ 0.07115986  0.06818835  0.07815106  0.07401947  0.38333893  0.06891692
   0.0807391   0.07364058  0.06936938  0.0697476 ]]
[[ 0.07101288  0.06804807  0.0779855   0.07386389  0.65985268  0.06877506
   0.08056521  0.07348577  0.06922562  0.069

   0.0673998   0.0579973   0.05793321  0.05518247]]
[[ 0.05600025  0.05602856  0.06109675  0.05805962  0.58729106  0.05657399
   0.06713233  0.05777498  0.05771329  0.05497411]]
[[ 0.05547148  0.05550193  0.06051098  0.05750693  0.76341569  0.05604118
   0.06647715  0.05722726  0.05716583  0.05445812]]
[[ 0.0541379   0.05416805  0.05903137  0.05611198  0.49226978  0.45438451
   0.06481735  0.05583818  0.05578275  0.05315268]]
[[ 0.05426165  0.05429017  0.05915974  0.05623663  0.44129527  0.05558975
   0.06495418  0.05596411  0.05590637  0.05327378]]
[[ 0.05449127  0.0545212   0.05941619  0.05647771  1.24952185  0.05582786
   0.06524102  0.05620372  0.05614648  0.05349959]]
[[ 0.05091075  0.05093982  0.05545072  0.0527358   0.70890349  0.05214123
   0.06080347  0.05248007  0.05243326  0.04999593]]
[[ 0.05003365  0.0500645   0.05448275  0.05182085  1.11044323  0.05124133
   0.05972284  0.05156976  0.05152662  0.04913779]]
[[ 0.04732888  0.04736081  0.05149691  0.04899962  0.50114685  0.0

   0.08417796  0.06819516  0.07064398  0.06807733]]
[[ 0.06851211  0.06738355  0.07365316  0.06945197  0.70567042  0.06907336
   0.08601777  0.0696051   0.07211541  0.06947854]]
[[ 0.06712576  0.0660294   0.0721348   0.06803977  0.57186788  0.06767206
   0.08416763  0.06818739  0.07063496  0.06806899]]
[[ 0.06629683  0.06521725  0.07123049  0.06719519 -0.01845683  0.06683453
   0.0830588   0.06733771  0.06975067  0.06722531]]
[[ 0.06790536  0.06679304  0.07298903  0.06883532 -0.01099003  0.06846047
   0.08520965  0.06898561  0.07146963  0.06886379]]
[[ 0.06936781  0.06822212  0.07459193  0.07032559  0.25089929  0.06993905
   0.08716851  0.07048189  0.07302994  0.07035138]]
[[ 0.06973339  0.06857929  0.07499149  0.07069807  0.04447395  0.0703071
   0.08766009  0.07085575  0.0734195   0.07072388]]
[[ 0.07085656  0.0696777   0.07622359  0.07184368  0.40164694  0.07144298
   0.08917282  0.07200699  0.0746216   0.07186703]]
[[ 0.07060216  0.06942768  0.07594308  0.07158352  0.3499712   0.07

[[ 0.08295806  0.08154815  0.08972522  0.08427978  0.09711334  0.08457305
   0.10642988  0.08450869  0.08794238  0.08963632]]
[[ 0.08317623  0.0817607   0.08997163  0.08450401  0.06908643  0.08479869
   0.10674687  0.08473528  0.08818069  0.08988125]]
[[ 0.0834416   0.08201994  0.09026939  0.08477581  0.34826174  0.08507242
   0.10713129  0.08500913  0.08846974  0.09017824]]
[[ 0.08316475  0.0817495   0.08995786  0.08449154 -0.19838175  0.08478646
   0.10673022  0.08472256  0.08816658  0.08986839]]
[[ 0.08394153  0.08250443  0.09082509  0.0852892   0.20137808  0.08558218
   0.10785943  0.08552418  0.08901223  0.09074065]]
[[ 0.0839358   0.08249911  0.09082178  0.08528378  0.4404614   0.08557935
   0.10785057  0.085519    0.08900839  0.09073313]]
[[ 0.08348763  0.08206435  0.09031907  0.08482351  0.24001363  0.08511944
   0.10719922  0.08505677  0.08851939  0.09023152]]
[[ 0.0834172   0.08199572  0.09024003  0.08475146 -0.05038175  0.08504719
   0.10709891  0.08498505  0.08844202  0.090

...

[[ 0.00333389  0.00339295  0.00354763  0.00368905  0.00358509  0.00360299
   1.0034101   0.0029903   0.00336459  0.00366145]]
[[ 0.00333151  0.00339051  0.00354514  0.0036864   0.00358252  0.00360046
   1.15005088  0.00298823  0.00336223  0.00365881]]
[[ 0.00331899  0.00337768  0.00353162  0.00367236  0.00356897  0.00358665
   1.26177955  0.00297705  0.00334948  0.00364491]]
[[ 0.00329871  0.00335708  0.00350999  0.00364976  0.00354711  0.00356468
   1.35008514  0.00295914  0.00332911  0.00362255]]
[[ 0.00327278  0.00333071  0.00348222  0.00362075  0.00351908  0.00353649
   0.64824075  0.00293617  0.00330292  0.00359384]]
[[ 0.00329441  0.00335272  0.00350547  0.00364495  0.00354252  0.0035601
   1.36071217  0.00295535  0.00332474  0.00361781]]
[[ 0.00326793  0.00332578  0.00347711  0.00361534  0.00351387  0.00353124
   0.88527864  0.00293185  0.00329806  0.00358844]]
[[ 0.00327356  1.05798888  0.00348312  0.00362164  0.00351992  0.00353738
   0.96939743  0.00293688  0.00330375  0.0035

[[ 0.00324644  0.00331896  0.00346102  0.00359922  0.0034903   0.0035076
   1.43339384  0.00291331  0.00328462  0.00357355]]
[[ 0.00321592  0.00328758  0.00342807  0.00356498  0.00345729  0.00347413
   1.19663036  0.00288613  0.00325357  0.00353966]]
[[ 0.00320107  0.00327251  0.00341238  0.00354845  0.00344134  0.00345822
   1.06232905  0.00287307  0.00323873  0.00352328]]
[[ 0.00319514  0.00326645  0.00340599  0.00354178  0.00343489  0.00345173
   0.70642948  0.00286782  0.00323269  0.00351668]]
[[ 0.00321217  0.00328382  0.00342423  0.00356087  0.00345331  0.00347022
   1.24467051  0.00288289  0.00324993  0.00353558]]
[[ 0.00319427  0.0032656   0.00340505  0.00354085  0.00343399  0.00345081
   1.08612978  0.00286708  0.00323187  0.0035158 ]]
[[ 0.00318687  0.00325795  0.00339715  0.00353254  0.003426    0.00344276
   1.24809062  0.00286049  0.00322434  0.00350755]]
[[ 0.00316928  0.00323999  0.00337819  0.00351278  0.00340688  0.00342355
   0.71048111  0.00284488  0.00320652  0.0034

[[ 0.00294222  0.00301361  0.00313455  0.00325127  0.00315476  0.00317587
   0.6279797   0.00263838  0.00297825  0.00322906]]
[[ 0.00296123  0.00303298  0.00315484  0.00327242  0.00317522  0.00319639
   1.26504517  0.00265511  0.00299736  0.00324997]]
[[ 0.00294501  0.00301643  0.00313748  0.00325431  0.00315773  0.00317884
   1.91671932  0.00264079  0.00298103  0.00323202]]
[[ 0.00289364  0.00296377  0.00308258  0.00319704  0.00310248  0.00312312
   1.10526323  0.00259531  0.00292904  0.0031753 ]]
[[ 0.00288662  0.0029566   0.00307507  0.00318923  0.00309492  0.00311551
   0.97300333  0.00258913  1.12457156  0.00316759]]
[[ 0.00289235  0.00296244  0.00308114  0.00319555  0.00310106  0.00312164
   0.94205678  0.00259423  0.00293393  0.00317384]]
[[ 0.00289387  0.00296403  0.00308276  0.00319727  0.00310271  0.0031233
   1.29302537  0.00259561  0.00293546  0.00317555]]
[[ 0.00287716  0.00294682  0.00306496  0.00317855  0.00308473  0.00310518
   0.79198998  0.00258073  0.00291844  0.0031

[[ 0.00277072  0.00284146  0.00294307  0.00305242  0.00295717  0.82630503
   0.97415394  0.0024765   0.00280205  0.00302611]]
[[ 0.002775    0.00284574  0.0029476   0.00305705  0.00296171  0.00298581
   0.76196301  0.00248027  0.00280625  0.00303067]]
[[ 0.00278535  0.00285633  0.00295861  0.00306856  1.03342617  0.00299695
   0.97401321  0.00248936  0.0028167   0.00304207]]
[[ 0.00279055  0.00286168  0.00296408  0.00307427  0.00298418  0.00300249
   0.56014824  0.00249399  0.00282194  0.00304772]]
[[ 0.00281115  0.00288272  0.00298588  0.00309706  0.0030062   0.00302468
   1.24895108  0.0025121   0.00284258  0.0030703 ]]
[[ 0.00279718  0.0028685   0.00297122  0.00308168  0.00299137  0.00300978
   0.38790911  0.00249989  0.00282866  0.00305509]]
[[ 0.00282646  0.00289852  0.00300245  0.00311422  0.00302276  0.00304143
   1.24829268  0.00252568  0.00285825  0.00308721]]
[[ 0.00281258  0.00288436  0.00298772  0.00309876  0.00300785  0.00302652
   1.43745828  0.00251349  0.00284426  0.003

   0.93796206  0.00247193  0.00280509  0.00301883]]
[[ 0.00277676  0.00286497  0.0029506   0.00304698  0.00296474  0.00297616
   0.91458875  0.00247346  0.00280685  0.00302078]]
[[ 0.0027796   0.00286794  0.0029537   0.00305017  0.00296782  0.00297928
   0.91393828  0.002476    0.0028098   0.00302391]]
[[ 0.0027825   0.94482678  0.00295661  0.00305334  0.00297088  0.00298223
   0.97406548  0.00247847  0.00281265  0.00302711]]
[[ 0.00278717  0.00288072  0.00296173  0.00305845  0.00297587  0.00298736
   1.37555146  0.00248267  0.00281736  0.0030321 ]]
[[ 0.00276739  0.00286033  0.00294048  0.00303659  0.00295463  0.00296599
   1.09257376  0.00246533  0.00279742  0.00301058]]
[[ 0.00276184  0.0028545   0.00293475  0.00303041  0.00294875  0.00296013
   0.76657957  0.00246037  0.00279181  0.00300435]]
[[ 0.00277175  0.00286483  0.00294526  0.00304144  0.00295935  0.00297078
   1.02007818  0.00246917  0.00280188  0.00301532]]
[[ 0.0027696   0.00286256  0.00294298  0.00303903  0.00295705  0.0

In [30]:
print(targetQ)

[[ 0.08601116  0.08637228  0.10096069  0.09053159  0.0920589   0.09787688
   0.09888906  0.15559117  0.10039488  0.09131332]]
